In [1]:
import pandas as pd
import numpy as np
import requests
import json
import zipfile
import xmltodict
import time
import warnings
import re
import xmltodict
from tqdm import tqdm
from io import BytesIO

warnings.filterwarnings(action='ignore')

# Data Load & Preprocess

## [1] DART

In [2]:
# api_key 세팅
api_key = 'a47ee3f06af0391401258e8fc94b90296fa2d7f5'

### 1. 고유번호

In [3]:
url = 'https://opendart.fss.or.kr/api/corpCode.xml'
res = requests.get(url, params={'crtfc_key': api_key})
z = zipfile.ZipFile(BytesIO(res.content))

data_xml = z.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data_dart_1 = pd.DataFrame(data_dict.get('result', {}).get('list'))
data_dart_1

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630
...,...,...,...,...
89448,01560457,삼원개발,None,20210506
89449,01556533,에이더블유파트너스,None,20210506
89450,01546299,동영와이케이,None,20210506
89451,00694605,미디어윌,None,20210506


In [4]:
#data_dart_1.to_pickle('data_dart_1.pkl')
data_dart_1 = pd.read_pickle('data_dart_1.pkl')
data_dart_1

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630
...,...,...,...,...
89361,01560457,삼원개발,None,20210506
89362,01556533,에이더블유파트너스,None,20210506
89363,01546299,동영와이케이,None,20210506
89364,00694605,미디어윌,None,20210506


In [5]:
# stock_code 있는 건들만 선별
data_dart_1_nn = data_dart_1[data_dart_1.stock_code.notnull()]
data_dart_1_nn

,corp_code,corp_name,stock_code,modify_date
2013,00260985,한빛네트,036720,20170630
2025,00264529,엔플렉스,040130,20170630
2026,00358545,동서정보기술,055000,20170630
2788,00231567,애드모바일,032600,20170630
3896,00247939,씨모스,037600,20170630
...,...,...,...,...
89302,00521390,뉴파워프라즈마,144960,20211125
89311,00664853,제이씨케미칼,137950,20211126
89313,00307028,경남제약,053950,20211201
89344,00874195,코썬바이오,204990,20211027


### 2. 기업개황

In [26]:
data_dart_2 = pd.DataFrame()

for code in tqdm(data_dart_1_nn.corp_code.tolist()):
    url = f'https://opendart.fss.or.kr/api/company.json?crtfc_key={api_key}&corp_code={code}'
    res = requests.get(url)
    text = json.loads(res.text)
    
    if text['message'] == '정상':
        data_dart_2 = pd.concat([data_dart_2, pd.DataFrame.from_dict(text, orient='index').T], axis=0)
    
data_dart_2

100%|██████████████████████████████████████████████████████████████████████████████| 3401/3401 [04:58<00:00, 11.41it/s]


,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
0,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
0,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
0,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
0,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,000,정상,00521390,(주)뉴파워프라즈마,"New Power Plasma Co.,Ltd",뉴파워프라즈마,144960,위순임,K,1101111784952,2128144110,경기도 수원시 권선구 산업로155번길 214 (주)뉴파워프라즈마,newpower.co.kr,,031-612-7600,031-612-7699,29271,19991007,12
0,000,정상,00664853,제이씨케미칼(주),"JC Chemical Corporation, LTD.",제이씨케미칼,137950,"윤사호, 이종응",K,1358110124726,1358185115,울산광역시 울주군 온산읍 화산1길 70,www.jcchemical.co.kr,,02-6321-1333,027615404,2049,20060324,12
0,000,정상,00307028,경남제약 주식회사,"KYUNG NAM PHARM.CO.,LTD.",경남제약,053950,오성원,K,1350110085255,1348142258,경상남도 의령군 동동리 1539-5,www.kyungnampharm.co.kr,,02-3490-5105,02)3490-5129,212,19981229,12
0,000,정상,00874195,(주)코썬바이오,"Kosun Bio Co., Ltd.",코썬바이오,204990,조규면,E,1101113505421,2148794321,서울특별시 영등포구 선유로 146 이앤씨드림타워 207호,www.hsvital.com,,02-2628-0550,02-2628-0561,10797,20060808,12


In [6]:
#data_dart_2.to_pickle('data_dart_2.pkl')
data_dart_2 = pd.read_pickle('data_dart_2.pkl')
data_dart_2

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
0,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
0,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
0,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
0,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,000,정상,00521390,(주)뉴파워프라즈마,"New Power Plasma Co.,Ltd",뉴파워프라즈마,144960,위순임,K,1101111784952,2128144110,경기도 수원시 권선구 산업로155번길 214 (주)뉴파워프라즈마,newpower.co.kr,,031-612-7600,031-612-7699,29271,19991007,12
0,000,정상,00664853,제이씨케미칼(주),"JC Chemical Corporation, LTD.",제이씨케미칼,137950,"윤사호, 이종응",K,1358110124726,1358185115,울산광역시 울주군 온산읍 화산1길 70,www.jcchemical.co.kr,,02-6321-1333,027615404,2049,20060324,12
0,000,정상,00307028,경남제약 주식회사,"KYUNG NAM PHARM.CO.,LTD.",경남제약,053950,오성원,K,1350110085255,1348142258,경상남도 의령군 동동리 1539-5,www.kyungnampharm.co.kr,,02-3490-5105,02)3490-5129,212,19981229,12
0,000,정상,00874195,(주)코썬바이오,"Kosun Bio Co., Ltd.",코썬바이오,204990,조규면,E,1101113505421,2148794321,서울특별시 영등포구 선유로 146 이앤씨드림타워 207호,www.hsvital.com,,02-2628-0550,02-2628-0561,10797,20060808,12


### 3. 직원현황

In [14]:
data_dart_3 = pd.DataFrame()

for code in tqdm(data_dart_1_nn.corp_code.tolist()):
    for year in ['2019', '2018']:
        url = f'https://opendart.fss.or.kr/api/empSttus.json?crtfc_key={api_key}&corp_code={code}&bsns_year={year}&reprt_code=11011'
        res = requests.get(url)
        text = json.loads(res.text)
        
        if text['message'] == '정상':
            data_dart_3 = pd.concat([data_dart_3, pd.DataFrame(text['list'])], axis=0)
        else:
            break
            
        time.sleep(0.5)
    
    time.sleep(1)

data_dart_3

100%|████████████████████████████████████████████████████████████████████████████| 3401/3401 [1:46:14<00:00,  1.87s/it]


,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,12,-,-,-,12,3.36,"865,006","72,084"
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,8,-,-,-,8,3.81,"395,183","49,398"
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,4.18,"308,347","77,087"
3,20200424000043,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,-,-,-,-,-,-,-,-
4,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,3.67,"619,224","88,461"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,20191001000296,K,00307028,경남제약,-,여,생산,-,-,-,54,-,3,-,57,15년11개월,"1,891,171,000","33,178,000"
0,20200331001563,E,00874195,코썬바이오,-,남,-,-,-,-,25,-,2,-,27,3년 10개월,"816,563,000","30,243,000"
1,20200331001563,E,00874195,코썬바이오,-,여,-,-,-,-,21,-,1,-,22,4년 6개월,"507,678,000","23,076,000"
0,20190401001362,E,00874195,코썬바이오,-,남,-,-,-,-,33,-,1,-,34,3년 3개월,"969,545,560","31,275,663"


In [7]:
# 2019, 2018
#data_dart_3.to_pickle('data_dart_3.pkl')
data_dart_3_1 = pd.read_pickle('data_dart_3.pkl')
data_dart_3_1

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,12,-,-,-,12,3.36,"865,006","72,084"
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,8,-,-,-,8,3.81,"395,183","49,398"
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,4.18,"308,347","77,087"
3,20200424000043,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,-,-,-,-,-,-,-,-
4,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,3.67,"619,224","88,461"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,20191001000296,K,00307028,경남제약,-,여,생산,-,-,-,54,-,3,-,57,15년11개월,"1,891,171,000","33,178,000"
0,20200331001563,E,00874195,코썬바이오,-,남,-,-,-,-,25,-,2,-,27,3년 10개월,"816,563,000","30,243,000"
1,20200331001563,E,00874195,코썬바이오,-,여,-,-,-,-,21,-,1,-,22,4년 6개월,"507,678,000","23,076,000"
0,20190401001362,E,00874195,코썬바이오,-,남,-,-,-,-,33,-,1,-,34,3년 3개월,"969,545,560","31,275,663"


In [8]:
#data_dart_3.to_pickle('data_dart_4.pkl')
# 2020, 2019
data_dart_3_2 = pd.read_pickle('data_dart_4.pkl')  
data_dart_3_2

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20210316000210,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,10,-,-,-,10,8.64,"943,156","94,316"
1,20210316000210,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,4,-,-,-,4,7.65,"319,674","79,919"
2,20210316000210,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,6.07,"521,380","130,345"
3,20210316000210,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,1,-,-,-,1,0.25,"13,514","13,514"
4,20210316000210,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,4.92,"853,502","121,929"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,-,여,제작,-,-,-,68,-,-,-,68,2.3,"1,953,000,000","32,000,000"
2,20210331003230,K,01264438,자이언트스텝,-,남,연구개발,-,-,-,14,-,-,-,14,1.8,"712,000,000","56,000,000"
3,20210331003230,K,01264438,자이언트스텝,-,여,연구개발,-,-,-,6,-,-,-,6,1.1,"146,000,000","30,000,000"
4,20210331003230,K,01264438,자이언트스텝,-,남,관리,-,-,-,11,2,-,-,13,4.1,"451,000,000","47,000,000"


In [150]:
data_dart_3 = pd.concat([data_dart_3_1, data_dart_3_2])
data_dart_3 = data_dart_3.drop_duplicates()
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,-,-,-,12,-,-,-,12,3.36,"865,006","72,084"
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,-,-,-,8,-,-,-,8,3.81,"395,183","49,398"
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,-,-,-,4,-,-,-,4,4.18,"308,347","77,087"
3,20200424000043,K,00956028,엑세스바이오,단위:USD,여,영업,-,-,-,-,-,-,-,-,-,-,-
4,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,-,-,-,7,-,-,-,7,3.67,"619,224","88,461"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,-,여,제작,-,-,-,68,-,-,-,68,2.3,"1,953,000,000","32,000,000"
2,20210331003230,K,01264438,자이언트스텝,-,남,연구개발,-,-,-,14,-,-,-,14,1.8,"712,000,000","56,000,000"
3,20210331003230,K,01264438,자이언트스텝,-,여,연구개발,-,-,-,6,-,-,-,6,1.1,"146,000,000","30,000,000"
4,20210331003230,K,01264438,자이언트스텝,-,남,관리,-,-,-,11,2,-,-,13,4.1,"451,000,000","47,000,000"


In [151]:
data_dart_3['year'] = data_dart_3.rcept_no.apply(lambda x: x[:4])
data_dart_3.year.value_counts()  # 2018, 2022 데이터 수가 확연히 적음 -> 삭제

2021    8809
2020    7975
2019    7394
2018     118
2022       4
Name: year, dtype: int64

In [152]:
# 2018, 2022 데이터 삭제
data_dart_3 = data_dart_3[~(data_dart_3.year.isin(['2018', '2022']))]
data_dart_3.year.value_counts()

2021    8809
2020    7975
2019    7394
Name: year, dtype: int64

In [153]:
# 년도별 기업 개수 파악
data_dart_3.pivot_table(index='year',
                        aggfunc=lambda x: len(x.unique()),
                        values='corp_code')

,corp_code
year,
2019,2243
2020,2367
2021,2505


In [154]:
# 기업 내 직군 개수 파악 -> maximum 18개 직군
data_dart_3.pivot_table(index='corp_code',
                        aggfunc=lambda x: len(x.unique()),
                        values='fo_bbm').sort_values(by='fo_bbm', ascending=False)

,fo_bbm
corp_code,
00453488,18
00992543,15
00126186,14
00132637,13
00375931,12
...,...
00311270,1
00310156,1
00307897,1


In [155]:
data_dart_3[data_dart_3.corp_code == '00453488']

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year
0,20200330003726,Y,00453488,다이나믹디자인,-,남,경영진,-,-,-,12,-,-,-,12,4.7,"1,181,000,000","98,000,000",2020
1,20200330003726,Y,00453488,다이나믹디자인,-,여,품질관리부문,-,-,-,1,-,-,-,1,3.9,"32,000,000","32,000,000",2020
2,20200330003726,Y,00453488,다이나믹디자인,-,남,R&D 센터,-,-,-,13,-,1,-,14,6.4,"716,000,000","51,000,000",2020
3,20200330003726,Y,00453488,다이나믹디자인,-,여,R&D 센터,-,-,-,1,1,-,-,2,15.8,"71,000,000","35,000,000",2020
4,20200330003726,Y,00453488,다이나믹디자인,-,남,해외사업부문,-,-,-,6,-,-,-,6,11.4,"341,000,000","57,000,000",2020
5,20200330003726,Y,00453488,다이나믹디자인,-,여,해외사업부문,-,-,-,1,-,-,-,1,6.8,"43,000,000","43,000,000",2020
6,20200330003726,Y,00453488,다이나믹디자인,-,여,경영진,-,-,-,-,-,-,-,-,-,-,-,2020
7,20200330003726,Y,00453488,다이나믹디자인,-,남,경영관리부문,-,-,-,17,-,-,5,22,3.9,"874,000,000","40,000,000",2020
8,20200330003726,Y,00453488,다이나믹디자인,-,여,경영관리부문,-,-,-,9,-,-,-,9,3.9,"359,000,000","40,000,000",2020
9,20200330003726,Y,00453488,다이나믹디자인,-,남,영업부문,-,-,-,11,-,-,-,11,6.5,"565,000,000","51,000,000",2020


In [156]:
# data preprocessing
# '-' 값을 null값으로 쓰고 있으므로, null로 전환 후 null값 현황 파악

for col in data_dart_3.columns:
    data_dart_3[col] = data_dart_3[col].apply(lambda x: np.NaN if x == '-' else x)
    
data_dart_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24178 entries, 0 to 5
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rcept_no                  24178 non-null  object 
 1   corp_cls                  24178 non-null  object 
 2   corp_code                 24178 non-null  object 
 3   corp_name                 24178 non-null  object 
 4   rm                        718 non-null    object 
 5   sexdstn                   24178 non-null  object 
 6   fo_bbm                    22528 non-null  object 
 7   reform_bfe_emp_co_rgllbr  0 non-null      float64
 8   reform_bfe_emp_co_cnttk   0 non-null      float64
 9   reform_bfe_emp_co_etc     0 non-null      float64
 10  rgllbr_co                 23465 non-null  object 
 11  rgllbr_abacpt_labrr_co    2806 non-null   object 
 12  cnttk_co                  11659 non-null  object 
 13  cnttk_abacpt_labrr_co     3012 non-null   object 
 14  sm        

In [157]:
# 모두 null인 컬럼 제외
# sm, avrg_cnwk_sdytrn, fyer_salary_totamt, jan_salary_am가 모두 null값인 케이스 제외

data_dart_3 = data_dart_3.drop(['reform_bfe_emp_co_rgllbr', 'reform_bfe_emp_co_cnttk', 'reform_bfe_emp_co_etc'], axis=1)
data_dart_3 = data_dart_3[~((data_dart_3.sm == np.NaN) &
                        (data_dart_3.avrg_cnwk_sdytrn == np.NaN) &
                        (data_dart_3.fyer_salary_totamt == np.NaN) &
                        (data_dart_3.jan_salary_am == np.NaN))]
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,NaN,12,3.36,"865,006","72,084",2020
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,NaN,8,3.81,"395,183","49,398",2020
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,NaN,4,4.18,"308,347","77,087",2020
3,20200424000043,K,00956028,엑세스바이오,단위:USD,여,영업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,NaN,7,3.67,"619,224","88,461",2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,NaN,68,2.3,"1,953,000,000","32,000,000",2021
2,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,NaN,14,1.8,"712,000,000","56,000,000",2021
3,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,NaN,6,1.1,"146,000,000","30,000,000",2021
4,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,NaN,13,4.1,"451,000,000","47,000,000",2021


In [158]:
# rm(비고) 중 단위 다른 케이스 살펴보기
data_dart_3[data_dart_3.rm.fillna('').str.contains('단위')]

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,NaN,12,3.36,"865,006","72,084",2020
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,NaN,8,3.81,"395,183","49,398",2020
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,NaN,4,4.18,"308,347","77,087",2020
3,20200424000043,K,00956028,엑세스바이오,단위:USD,여,영업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,NaN,7,3.67,"619,224","88,461",2020
5,20200424000043,K,00956028,엑세스바이오,단위:USD,여,연구,3,NaN,NaN,NaN,3,4.57,"180,408","60,136",2020
6,20200424000043,K,00956028,엑세스바이오,단위:USD,남,생산,19,NaN,NaN,NaN,19,3.12,"1,121,119","59,006",2020
7,20200424000043,K,00956028,엑세스바이오,단위:USD,여,생산,9,NaN,NaN,NaN,9,5.44,"536,325","59,592",2020
0,20190430001200,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,NaN,12,3.0,"822,894","68,575",2019
1,20190430001200,K,00956028,엑세스바이오,단위:USD,여,사무,7,NaN,NaN,NaN,7,4.0,"388,021","55,432",2019


In [159]:
# fyer_salary_totamt, jan_salary_am 이상한 케이스 발견 -> 삭제

print(data_dart_3.shape[0])
data_dart_3 = data_dart_3[~((data_dart_3.corp_code == '00261957') & (data_dart_3.year == '2019'))]
print(data_dart_3.shape[0])

24178
24176


In [160]:
# USD 기준 급여 -> 원 기준으로 통일

data_dart_3['fyer_salary_totamt_rev'] = data_dart_3.fyer_salary_totamt.fillna('-1').str.replace(',', '').astype(np.int64)
data_dart_3['fyer_salary_totamt_rev'] = data_dart_3.fyer_salary_totamt_rev.apply(lambda x: np.NaN if x == -1 else x)

data_dart_3['jan_salary_am_rev'] = data_dart_3.jan_salary_am.fillna('-1').str.replace(',', '').astype(np.int64)
data_dart_3['jan_salary_am_rev'] = data_dart_3.jan_salary_am_rev.apply(lambda x: np.NaN if x == -1 else x)

# 1$ = 1,200원으로 가정하고 변환
data_dart_3['fyer_salary_totamt_rev2'] = data_dart_3.apply(lambda r: r.fyer_salary_totamt_rev if r.rm is np.NaN
                  #else r.fyer_salary_totamt_rev * 1200 * 1000 if '천USD' in r.rm 
                  else r.fyer_salary_totamt_rev * 1200 if 'USD' in r.rm 
                  else r.fyer_salary_totamt_rev, axis=1)
data_dart_3['jan_salary_am_rev2'] = data_dart_3.apply(lambda r: r.jan_salary_am_rev if r.rm is np.NaN
                  #else r.jan_salary_am_rev * 1200 * 1000 if '천USD' in r.rm 
                  else r.jan_salary_am_rev * 1200 if 'USD' in r.rm 
                  else r.jan_salary_am_rev, axis=1)
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,NaN,12,3.36,"865,006","72,084",2020,8.650060e+05,72084.0,1.038007e+09,86500800.0
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,NaN,8,3.81,"395,183","49,398",2020,3.951830e+05,49398.0,4.742196e+08,59277600.0
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,NaN,4,4.18,"308,347","77,087",2020,3.083470e+05,77087.0,3.700164e+08,92504400.0
3,20200424000043,K,00956028,엑세스바이오,단위:USD,여,영업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,NaN,NaN,NaN,NaN
4,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,NaN,7,3.67,"619,224","88,461",2020,6.192240e+05,88461.0,7.430688e+08,106153200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,NaN,68,2.3,"1,953,000,000","32,000,000",2021,1.953000e+09,32000000.0,1.953000e+09,32000000.0
2,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,NaN,14,1.8,"712,000,000","56,000,000",2021,7.120000e+08,56000000.0,7.120000e+08,56000000.0
3,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,NaN,6,1.1,"146,000,000","30,000,000",2021,1.460000e+08,30000000.0,1.460000e+08,30000000.0
4,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,NaN,13,4.1,"451,000,000","47,000,000",2021,4.510000e+08,47000000.0,4.510000e+08,47000000.0


In [161]:
# fo_bbm 명칭에서 공백 제거

data_dart_3['fo_bbm'] = data_dart_3.fo_bbm.fillna('').str.replace('\s+', '', regex=True)
data_dart_3['fo_bbm'].unique().tolist()[:10]

['사무', '영업', '연구', '생산', '관리사무직', '', '전사', '관리', '재무', '기타']

In [162]:
# 평균 근속 년수 데이터 정제
# x년 xx개월 케이스 살펴보기
data_dart_3[data_dart_3.avrg_cnwk_sdytrn.fillna('').str.contains('년')].head(10)

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2
0,20200327002143,E,00232317,지오엠씨,NaN,남,관리사무직,8,NaN,1,NaN,9,1년2개월,"257,388,809","28,598,757",2020,2.573888e+08,28598757.0,2.573888e+08,28598757.0
1,20200327002143,E,00232317,지오엠씨,NaN,여,관리사무직,3,NaN,NaN,NaN,3,5년0개월,"71,282,353","23,760,784",2020,7.128235e+07,23760784.0,7.128235e+07,23760784.0
1,20190401002398,E,00232317,지오엠씨,NaN,여,관리사무직,2,NaN,NaN,NaN,2,6년 2개월,"63,269,832","31,634,916",2019,6.326983e+07,31634916.0,6.326983e+07,31634916.0
0,20200330001373,E,01063963,유쎌,NaN,남,,7,NaN,NaN,NaN,7,5년5개월,"468,800,000","66,971,000",2020,4.688000e+08,66971000.0,4.688000e+08,66971000.0
1,20200330001373,E,01063963,유쎌,NaN,여,,15,NaN,NaN,NaN,15,5년8개월,"385,267,000","25,684,000",2020,3.852670e+08,25684000.0,3.852670e+08,25684000.0
0,20190329001523,E,01063963,유쎌,NaN,남,화장품,8,NaN,NaN,NaN,8,5년 1개월,"502,071,000","62,759,000",2019,5.020710e+08,62759000.0,5.020710e+08,62759000.0
1,20190329001523,E,01063963,유쎌,NaN,여,화장품,37,NaN,NaN,NaN,37,3년 9개월,"656,966,000","17,756,000",2019,6.569660e+08,17756000.0,6.569660e+08,17756000.0
0,20200330004519,E,00343127,자유투어,NaN,남,여행사업부,47,47,NaN,NaN,47,4년5개월,"1,793,156,000","38,152,000",2020,1.793156e+09,38152000.0,1.793156e+09,38152000.0
1,20200330004519,E,00343127,자유투어,NaN,여,여행사업부,85,85,NaN,NaN,85,3년2개월,"2,621,525,000","30,841,000",2020,2.621525e+09,30841000.0,2.621525e+09,30841000.0
0,20190710000677,E,00343127,자유투어,NaN,남,여행사업부,49,NaN,1,NaN,50,4년 4개월,"1,876,003,000","38,268,000",2019,1.876003e+09,38268000.0,1.876003e+09,38268000.0


In [179]:
# x년 xx개월 -> float 형태로 수정
# xYx -> float 형태로 수정
# xx일은 무시

data_dart_3['avrg_cnwk_sdytrn_rev'] = data_dart_3.avrg_cnwk_sdytrn.apply(
    lambda x: x if type(x) == float 
    else int(re.search('(\d+)\s*년\s*(\d+)\s*개?월' , x).group(1)) + 
                  round(int(re.search('(\d+)\s*년\s*(\d+)\s*개?월' , x).group(2))/12, 2) if '년' in x and '월' in x
    else int(re.search('(\d+)\s*년\s*' , x).group(1)) if '년' in x
    else round(int(re.search('(\d+)\s*개?월' , x).group(1))/12, 2) if '월' in x
    else int(re.search('(\d+)\s*Y\s*(\d+)' , x).group(1)) + 
         round(int(re.search('(\d+)\s*Y\s*(\d+)' , x).group(2))/12, 2) if 'Y' in x
    else x)

data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,...,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,avrg_cnwk_sdytrn_rev
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,...,12,3.36,"865,006","72,084",2020,8.650060e+05,72084.0,1.038007e+09,86500800.0,3.36
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,...,8,3.81,"395,183","49,398",2020,3.951830e+05,49398.0,4.742196e+08,59277600.0,3.81
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,...,4,4.18,"308,347","77,087",2020,3.083470e+05,77087.0,3.700164e+08,92504400.0,4.18
3,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,...,7,3.67,"619,224","88,461",2020,6.192240e+05,88461.0,7.430688e+08,106153200.0,3.67
4,20200424000043,K,00956028,엑세스바이오,단위:USD,여,연구,3,NaN,NaN,...,3,4.57,"180,408","60,136",2020,1.804080e+05,60136.0,2.164896e+08,72163200.0,4.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16122,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,...,68,2.3,"1,953,000,000","32,000,000",2021,1.953000e+09,32000000.0,1.953000e+09,32000000.0,2.3
16123,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,...,14,1.8,"712,000,000","56,000,000",2021,7.120000e+08,56000000.0,7.120000e+08,56000000.0,1.8
16124,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,...,6,1.1,"146,000,000","30,000,000",2021,1.460000e+08,30000000.0,1.460000e+08,30000000.0,1.1
16125,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,...,13,4.1,"451,000,000","47,000,000",2021,4.510000e+08,47000000.0,4.510000e+08,47000000.0,4.1


In [180]:
data_dart_3[data_dart_3.avrg_cnwk_sdytrn.fillna('').str.contains('년')].head(10)

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,...,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,avrg_cnwk_sdytrn_rev


In [183]:
data_dart_3['avrg_cnwk_sdytrn_rev'].unique()

array([ 3.3 ,  3.8 ,  4.1 ,  3.6 ,  4.5 ,  3.1 ,  5.4 ,  3.  ,  4.  ,
        5.  ,  0.  ,  7.  ,  1.  ,  2.  ,  1.8 ,  1.6 ,  1.2 ,  0.9 ,
        0.7 , 30.  , 20.  , 11.  , 10.  ,  8.5 , 14.6 , 14.7 ,  4.8 ,
        4.6 ,  8.  , 10.5 , 14.8 , 13.7 ,  4.4 ,  4.3 ,  0.8 ,  6.3 ,
        5.1 ,  5.8 ,  5.3 ,  3.2 ,  1.3 ,  2.2 ,  2.4 ,  6.  , 13.  ,
       12.1 , 12.  ,  6.8 ,  4.7 ,  2.6 ,  1.5 , 14.9 , 11.8 ,  9.7 ,
       11.3 ,  8.8 , 15.  ,  9.4 , 11.9 , 13.8 , 12.2 ,  2.9 ,  2.1 ,
       11.7 ,  9.  ,  7.9 ,  3.5 ,  1.9 ,  6.4 ,  7.1 ,  2.8 , 31.  ,
       37.  , 18.  , 10.3 , 12.8 , 10.4 ,  3.4 ,  2.3 ,  3.9 , 24.  ,
       22.  ,  2.7 ,  2.5 , 23.  , 19.9 ,  9.9 , 18.9 ,  8.9 ,  4.9 ,
       12.6 , 10.2 , 17.4 , 13.6 , 12.9 ,  9.2 ,  7.2 , 17.  ,  6.2 ,
        5.2 ,  5.9 ,  6.1 ,  6.5 ,  8.3 ,  3.7 ,  8.2 , 11.1 ,  8.6 ,
        6.6 ,  6.7 ,  9.8 , 14.  , 19.  ,  5.7 ,  9.1 ,  7.6 ,  1.7 ,
        5.6 ,  8.7 , 13.1 , 17.7 ,  0.5 , 16.9 , 14.3 , 16.  ,  1.4 ,
        1.1 ,  5.5 ,

In [182]:
# 추가로 null 케이스 제외 및 index reset

data_dart_3 = data_dart_3[data_dart_3['avrg_cnwk_sdytrn_rev'].notnull()]
data_dart_3 = data_dart_3[(data_dart_3['avrg_cnwk_sdytrn_rev'] != '남') & (data_dart_3['avrg_cnwk_sdytrn_rev'] != '여')]
data_dart_3['avrg_cnwk_sdytrn_rev'] = data_dart_3['avrg_cnwk_sdytrn_rev'].str.replace('-', '.')
data_dart_3['avrg_cnwk_sdytrn_rev'] = data_dart_3['avrg_cnwk_sdytrn_rev'].str.replace(',', '.')
data_dart_3['avrg_cnwk_sdytrn_rev'] = data_dart_3['avrg_cnwk_sdytrn_rev'].str.replace('\s+', '', regex=True)
data_dart_3['avrg_cnwk_sdytrn_rev'] = data_dart_3['avrg_cnwk_sdytrn_rev'].str.replace('.$', '', regex=True)
data_dart_3.loc[data_dart_3['avrg_cnwk_sdytrn_rev'] == '', 'avrg_cnwk_sdytrn_rev'] = '0'
data_dart_3['avrg_cnwk_sdytrn_rev'] = data_dart_3['avrg_cnwk_sdytrn_rev'].astype(float)
#data_dart_3 = data_dart_3[data_dart_3['avrg_cnwk_sdytrn_rev'] != 0]
data_dart_3.reset_index(drop=True, inplace=True)
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,...,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,avrg_cnwk_sdytrn_rev
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,...,12,3.36,"865,006","72,084",2020,8.650060e+05,72084.0,1.038007e+09,86500800.0,3.3
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,...,8,3.81,"395,183","49,398",2020,3.951830e+05,49398.0,4.742196e+08,59277600.0,3.8
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,...,4,4.18,"308,347","77,087",2020,3.083470e+05,77087.0,3.700164e+08,92504400.0,4.1
3,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,...,7,3.67,"619,224","88,461",2020,6.192240e+05,88461.0,7.430688e+08,106153200.0,3.6
4,20200424000043,K,00956028,엑세스바이오,단위:USD,여,연구,3,NaN,NaN,...,3,4.57,"180,408","60,136",2020,1.804080e+05,60136.0,2.164896e+08,72163200.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16122,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,...,68,2.3,"1,953,000,000","32,000,000",2021,1.953000e+09,32000000.0,1.953000e+09,32000000.0,2.0
16123,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,...,14,1.8,"712,000,000","56,000,000",2021,7.120000e+08,56000000.0,7.120000e+08,56000000.0,1.0
16124,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,...,6,1.1,"146,000,000","30,000,000",2021,1.460000e+08,30000000.0,1.460000e+08,30000000.0,1.0
16125,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,...,13,4.1,"451,000,000","47,000,000",2021,4.510000e+08,47000000.0,4.510000e+08,47000000.0,4.0


In [184]:
len(data_dart_3.corp_code.unique())

1778

In [191]:
data_dart_3['fyer_salary_totamt_rev2'] / 1000

0        1038007.2
1         474219.6
2         370016.4
3         743068.8
4         216489.6
           ...    
16122    1953000.0
16123     712000.0
16124     146000.0
16125     451000.0
16126     291000.0
Name: fyer_salary_totamt_rev2, Length: 16127, dtype: float64

In [195]:
# 급여 이상한 케이스 변경 (평균 100억 이상 수급)
# 이 외에도 데이터 이상한 케이스 있지만, 다 제거하기에는 데이터 자체 문제인 걸로 보임..

data_dart_3['fyer_salary_totamt_rev2'] = \
    data_dart_3.apply(lambda r: r['fyer_salary_totamt_rev2'] / 1000 if r['jan_salary_am_rev2'] >= 10000000000 
                      else r['fyer_salary_totamt_rev2'], 
                      axis=1) 
data_dart_3['jan_salary_am_rev2'] = \
    data_dart_3.jan_salary_am_rev2.apply(lambda r: r / 1000 if r >= 10000000000 
                      else r)
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,...,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,avrg_cnwk_sdytrn_rev
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,...,12,3.36,"865,006","72,084",2020,8.650060e+05,72084.0,1.038007e+09,86500800.0,3.3
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,...,8,3.81,"395,183","49,398",2020,3.951830e+05,49398.0,4.742196e+08,59277600.0,3.8
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,...,4,4.18,"308,347","77,087",2020,3.083470e+05,77087.0,3.700164e+08,92504400.0,4.1
3,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,...,7,3.67,"619,224","88,461",2020,6.192240e+05,88461.0,7.430688e+08,106153200.0,3.6
4,20200424000043,K,00956028,엑세스바이오,단위:USD,여,연구,3,NaN,NaN,...,3,4.57,"180,408","60,136",2020,1.804080e+05,60136.0,2.164896e+08,72163200.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16122,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,...,68,2.3,"1,953,000,000","32,000,000",2021,1.953000e+09,32000000.0,1.953000e+09,32000000.0,2.0
16123,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,...,14,1.8,"712,000,000","56,000,000",2021,7.120000e+08,56000000.0,7.120000e+08,56000000.0,1.0
16124,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,...,6,1.1,"146,000,000","30,000,000",2021,1.460000e+08,30000000.0,1.460000e+08,30000000.0,1.0
16125,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,...,13,4.1,"451,000,000","47,000,000",2021,4.510000e+08,47000000.0,4.510000e+08,47000000.0,4.0


In [9]:
#data_dart_3.to_pickle('data_dart_3_rev.pkl')
data_dart_3 = pd.read_pickle('data_dart_3_rev.pkl')
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,...,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,avrg_cnwk_sdytrn_rev
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,...,12,3.36,"865,006","72,084",2020,8.650060e+05,72084.0,1.038007e+09,86500800.0,3.3
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,...,8,3.81,"395,183","49,398",2020,3.951830e+05,49398.0,4.742196e+08,59277600.0,3.8
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,...,4,4.18,"308,347","77,087",2020,3.083470e+05,77087.0,3.700164e+08,92504400.0,4.1
3,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,...,7,3.67,"619,224","88,461",2020,6.192240e+05,88461.0,7.430688e+08,106153200.0,3.6
4,20200424000043,K,00956028,엑세스바이오,단위:USD,여,연구,3,NaN,NaN,...,3,4.57,"180,408","60,136",2020,1.804080e+05,60136.0,2.164896e+08,72163200.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16122,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,...,68,2.3,"1,953,000,000","32,000,000",2021,1.953000e+09,32000000.0,1.953000e+09,32000000.0,2.0
16123,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,...,14,1.8,"712,000,000","56,000,000",2021,7.120000e+08,56000000.0,7.120000e+08,56000000.0,1.0
16124,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,...,6,1.1,"146,000,000","30,000,000",2021,1.460000e+08,30000000.0,1.460000e+08,30000000.0,1.0
16125,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,...,13,4.1,"451,000,000","47,000,000",2021,4.510000e+08,47000000.0,4.510000e+08,47000000.0,4.0


### 4. 주요계정
* 사업보고서만 추출

In [215]:
data_dart_4 = pd.DataFrame()
year = '2020'

for code in tqdm(data_dart_3.corp_code.unique().tolist()):
    url = f'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key={api_key}&corp_code={code}&bsns_year={year}&reprt_code=11011'
    res = requests.get(url)
    text = json.loads(res.text)
    if text['message'] == '정상':
        # 손익계산서만 추출
        tmp = pd.DataFrame([d for d in text['list'] if d['sj_div'] == 'IS'])
        data_dart_4 = pd.concat([data_dart_4, tmp], axis=0)
    
    time.sleep(0.5)
    
data_dart_4

100%|██████████████████████████████████████████████████████████████████████████████| 2587/2587 [26:38<00:00,  1.62it/s]


,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"103,213,745",제 18 기,2019.01.01 ~ 2019.12.31,"36,939,637",제 17 기,2018.01.01 ~ 2018.12.31,"36,571,123",23
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,"58,222,516",제 18 기,2019.01.01 ~ 2019.12.31,"-6,895,892",제 17 기,2018.01.01 ~ 2018.12.31,"-3,982,259",25
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,"52,946,039",제 18 기,2019.01.01 ~ 2019.12.31,"-18,540,180",제 17 기,2018.01.01 ~ 2018.12.31,"-5,580,613",27
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,"40,517,092",제 18 기,2019.01.01 ~ 2019.12.31,"-17,720,599",제 17 기,2018.01.01 ~ 2018.12.31,"-5,802,229",29
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"92,215,510",제 18 기,2019.01.01 ~ 2019.12.31,"32,067,422",제 17 기,2018.01.01 ~ 2018.12.31,"34,158,734",24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20210331003230,11011,2020,01264438,289220,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 13 기,2020.01.01 ~ 2020.12.31,"-1,459,877,388",제 12 기,2019.01.01 ~ 2019.12.31,"-1,779,724,876",제 11 기,2018.01.01 ~ 2018.12.31,-,27
2,20210331003230,11011,2020,01264438,289220,CFS,연결재무제표,IS,손익계산서,당기순이익,제 13 기,2020.01.01 ~ 2020.12.31,"-1,422,167,643",제 12 기,2019.01.01 ~ 2019.12.31,"-3,142,183,408",제 11 기,2018.01.01 ~ 2018.12.31,-,29
3,20210331003230,11011,2020,01264438,289220,OFS,재무제표,IS,손익계산서,매출액,제 13 기,2020.01.01 ~ 2020.12.31,"15,535,548,208",제 12 기,2019.01.01 ~ 2019.12.31,"12,839,024,832",제 11 기,2018.01.01 ~ 2018.12.31,-,24
4,20210331003230,11011,2020,01264438,289220,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 13 기,2020.01.01 ~ 2020.12.31,"-1,506,710,609",제 12 기,2019.01.01 ~ 2019.12.31,"-3,895,408,874",제 11 기,2018.01.01 ~ 2018.12.31,-,28


In [10]:
#data_dart_4.to_pickle('data_dart_5.pkl')
data_dart_4 = pd.read_pickle('data_dart_5.pkl')  
data_dart_4.reset_index(inplace=True, drop=True)
data_dart_4

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"103,213,745",제 18 기,2019.01.01 ~ 2019.12.31,"36,939,637",제 17 기,2018.01.01 ~ 2018.12.31,"36,571,123",23
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,"58,222,516",제 18 기,2019.01.01 ~ 2019.12.31,"-6,895,892",제 17 기,2018.01.01 ~ 2018.12.31,"-3,982,259",25
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,"52,946,039",제 18 기,2019.01.01 ~ 2019.12.31,"-18,540,180",제 17 기,2018.01.01 ~ 2018.12.31,"-5,580,613",27
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,"40,517,092",제 18 기,2019.01.01 ~ 2019.12.31,"-17,720,599",제 17 기,2018.01.01 ~ 2018.12.31,"-5,802,229",29
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"92,215,510",제 18 기,2019.01.01 ~ 2019.12.31,"32,067,422",제 17 기,2018.01.01 ~ 2018.12.31,"34,158,734",24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15366,20210331003230,11011,2020,01264438,289220,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 13 기,2020.01.01 ~ 2020.12.31,"-1,459,877,388",제 12 기,2019.01.01 ~ 2019.12.31,"-1,779,724,876",제 11 기,2018.01.01 ~ 2018.12.31,-,27
15367,20210331003230,11011,2020,01264438,289220,CFS,연결재무제표,IS,손익계산서,당기순이익,제 13 기,2020.01.01 ~ 2020.12.31,"-1,422,167,643",제 12 기,2019.01.01 ~ 2019.12.31,"-3,142,183,408",제 11 기,2018.01.01 ~ 2018.12.31,-,29
15368,20210331003230,11011,2020,01264438,289220,OFS,재무제표,IS,손익계산서,매출액,제 13 기,2020.01.01 ~ 2020.12.31,"15,535,548,208",제 12 기,2019.01.01 ~ 2019.12.31,"12,839,024,832",제 11 기,2018.01.01 ~ 2018.12.31,-,24
15369,20210331003230,11011,2020,01264438,289220,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 13 기,2020.01.01 ~ 2020.12.31,"-1,506,710,609",제 12 기,2019.01.01 ~ 2019.12.31,"-3,895,408,874",제 11 기,2018.01.01 ~ 2018.12.31,-,28


In [237]:
data_dart_4_1 = pd.concat([data_dart_4.iloc[:, :10], data_dart_4.iloc[:, 10:13]], axis=1)
data_dart_4_1.columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
                         'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'nm', 'dt', 'amount']
data_dart_4_2 = pd.concat([data_dart_4.iloc[:, :10], data_dart_4.iloc[:, 13:16]], axis=1)
data_dart_4_2.columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
                         'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'nm', 'dt', 'amount']
data_dart_4_3 = pd.concat([data_dart_4.iloc[:, :10], data_dart_4.iloc[:, 16:19]], axis=1)
data_dart_4_3.columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
                         'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'nm', 'dt', 'amount']

data_dart_4_rev = pd.concat([data_dart_4_1, data_dart_4_2, data_dart_4_3])
data_dart_4_rev

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,nm,dt,amount
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"103,213,745"
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,"58,222,516"
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,"52,946,039"
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,"40,517,092"
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"92,215,510"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15366,20210331003230,11011,2020,01264438,289220,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 11 기,2018.01.01 ~ 2018.12.31,-
15367,20210331003230,11011,2020,01264438,289220,CFS,연결재무제표,IS,손익계산서,당기순이익,제 11 기,2018.01.01 ~ 2018.12.31,-
15368,20210331003230,11011,2020,01264438,289220,OFS,재무제표,IS,손익계산서,매출액,제 11 기,2018.01.01 ~ 2018.12.31,-
15369,20210331003230,11011,2020,01264438,289220,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 11 기,2018.01.01 ~ 2018.12.31,-


In [242]:
# 금액 정보 없는 데이터 제거
# 금액 데이터 타입 변경

data_dart_4_rev = data_dart_4_rev[data_dart_4_rev.amount != '-']
data_dart_4_rev['amount'] = data_dart_4_rev['amount'].str.replace(',', '').astype(np.int64)
data_dart_4_rev

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,nm,dt,amount
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,103213745
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,58222516
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,52946039
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,40517092
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,92215510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15360,20210329001000,11011,2020,00140566,002680,OFS,재무제표,IS,손익계산서,당기순이익,제 60 기,2018.01.01 ~ 2018.12.31,-3723580588
15361,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,매출액,제 8 기,2018.01.01 ~ 2018.12.31,276496903
15362,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,영업이익,제 8 기,2018.01.01 ~ 2018.12.31,-3375624884
15363,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 8 기,2018.01.01 ~ 2018.12.31,-7344671390


In [11]:
#data_dart_4_rev.reset_index(inplace=True, drop=True)
#data_dart_4_rev.to_pickle('data_dart_5_rev.pkl')
data_dart_4_rev = pd.read_pickle('data_dart_5_rev.pkl') 
data_dart_4_rev

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,nm,dt,amount
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,103213745
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,58222516
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,52946039
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,40517092
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,92215510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45725,20210329001000,11011,2020,00140566,002680,OFS,재무제표,IS,손익계산서,당기순이익,제 60 기,2018.01.01 ~ 2018.12.31,-3723580588
45726,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,매출액,제 8 기,2018.01.01 ~ 2018.12.31,276496903
45727,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,영업이익,제 8 기,2018.01.01 ~ 2018.12.31,-3375624884
45728,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 8 기,2018.01.01 ~ 2018.12.31,-7344671390


## [2] 공공데이터포털

In [12]:
api_key2 = 'Ql1s+2/whY32HrSPdqi2pSF9PU386zmSFS8FXVgUuPDyZKezjhmtjAOzu5fSh6D9F27wJvrwyeB+iHgHo7V+Xg=='

### 1. 국민연금 가입 사업장 내역 - 사업장 정보조회 서비스

In [255]:
corp_name_list = data_dart_1[data_dart_1.corp_code.isin(data_dart_4.corp_code.unique().tolist())].corp_name.tolist()
corp_name_list[:10]

['엑세스바이오',
 '글로벌에스엠',
 '씨케이에이치',
 'GRT',
 '윙입푸드',
 '로스웰',
 '골든센츄리',
 '코오롱티슈진',
 'SBI핀테크솔루션즈',
 '영신금속']

In [309]:
data_ins_1 = pd.DataFrame()
url = 'http://apis.data.go.kr/B552015/NpsBplcInfoInqireService/getBassInfoSearch'
cnt = 0

for corp_name in tqdm(corp_name_list[1948:]):
    params ={'serviceKey': api_key2, 'wkpl_nm': corp_name}
    response = requests.get(url, params=params)

    cc = xmltodict.parse(response.content)  # return collections.OrderedDict
    dd = json.loads(json.dumps(cc)) # return dict
    
    try:
        if dd['response']['header']['resultCode'] == '00':
            if dd['response']['body']['totalCount'] != '0':
                data_ins_1 = pd.concat([data_ins_1,
                                        pd.DataFrame(dd['response']['body']['items']['item']).sort_values(
                                            by='dataCrtYm', ascending=False).head(1)], axis=0)
    except:
        cnt += 1
        print(f'{cnt} {corp_name}')

    
data_ins_1

100%|██████████████████████████████████████████████████████████████████████████████| 201/201 [1:11:53<00:00, 21.46s/it]


,bzowrRgstNo,dataCrtYm,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,seq,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,135813****,202104,41,550,360,32344013,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
1,653880****,202105,11,440,123,33251682,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
5,114871****,202103,11,680,107,31966734,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
6,410814****,202112,29,170,129,36603791,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
5,107811****,202112,47,111,111,36654621,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...
5,609812****,202112,48,123,128,36655599,1,테라사이언스(주),경상남도 창원시 성산구 창원대로1144번길,1
5,126815****,202112,11,500,105,36959120,1,"(주)파이런텍/일용/(주)뉴파워프라즈마 마곡 신기술센타 신축공사 중 전기,통신,소방...",None,1
2,129866****,202011,11,680,115,28756771,1,에스제이씨케미칼(주),서울특별시 강남구 밤고개로1길,1
7,505852****,202107,11,680,107,34283750,1,경남제약헬스케어(주)도산돼지,서울특별시 강남구 선릉로153길,1


In [330]:
# dict 항목이 하나인 케이스들만 다시 데이터 수집

data_ins_1_err = pd.DataFrame()
url = 'http://apis.data.go.kr/B552015/NpsBplcInfoInqireService/getBassInfoSearch'
cnt = 0

for corp_name in tqdm(['LG디스플레이', '미투젠', '아바텍', 'DL건설', '한창바이오텍', '디티알오토모티브']):
    params ={'serviceKey': api_key2, 'wkpl_nm': corp_name}
    response = requests.get(url, params=params)

    cc = xmltodict.parse(response.content)  # return collections.OrderedDict
    dd = json.loads(json.dumps(cc)) # return dict
    

    if dd['response']['header']['resultCode'] == '00':
        if dd['response']['body']['totalCount'] != '0':
            data_ins_1_err = pd.concat([data_ins_1_err,
                                    pd.DataFrame([dd['response']['body']['items']['item']]).sort_values(
                                        by='dataCrtYm', ascending=False).head(1)], axis=0)


data_ins_1_err

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:18<00:00, 23.03s/it]


,bzowrRgstNo,dataCrtYm,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,seq,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,229813****,202112,41,480,310,36996058,1,백상엘앤씨（주）/(일용)LG디스플레이 파주 조경관리 용역(2021년),경기도 파주시 월롱면 엘지로,1
0,111843****,202112,11,680,108,36902629,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
0,318810****,201511,43,730,250,20310715,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
0,433880****,202110,48,310,370,35940676,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
0,139810****,202112,11,545,101,36501345,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1
0,278850****,202112,31,710,250,37039262,1,주식회사디티알오토모티브울산2공장,울산광역시 울주군 온산읍 화산2길,1


In [ ]:
'''
1 LG디스플레이
2 미투젠
3 아바텍
4 DL건설
5 한창바이오텍
6 디티알오토모티브
1948/2149
'''

In [335]:
data_ins_1 = pd.concat([data_ins_1, data_ins_1_err]).drop_duplicates().reset_index(drop=True)
data_ins_1

,bzowrRgstNo,dataCrtYm,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,seq,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,135813****,202104,41,550,360,32344013,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
1,653880****,202105,11,440,123,33251682,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
2,114871****,202103,11,680,107,31966734,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
3,410814****,202112,29,170,129,36603791,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
4,107811****,202112,47,111,111,36654621,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...
1981,111843****,202112,11,680,108,36902629,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
1982,318810****,201511,43,730,250,20310715,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
1983,433880****,202110,48,310,370,35940676,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
1984,139810****,202112,11,545,101,36501345,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1


In [13]:
#data_ins_1.to_pickle('data_ins_1.pkl')
data_ins_1 = pd.read_pickle('data_ins_1.pkl')
data_ins_1

,bzowrRgstNo,dataCrtYm,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,seq,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,135813****,202104,41,550,360,32344013,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
1,653880****,202105,11,440,123,33251682,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
2,114871****,202103,11,680,107,31966734,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
3,410814****,202112,29,170,129,36603791,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
4,107811****,202112,47,111,111,36654621,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...
1981,111843****,202112,11,680,108,36902629,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
1982,318810****,201511,43,730,250,20310715,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
1983,433880****,202110,48,310,370,35940676,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
1984,139810****,202112,11,545,101,36501345,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1


### 2. 국민연금 가입 사업장 내역 - 상세 정보조회 

In [348]:
data_ins_2 = pd.DataFrame()
url = 'http://apis.data.go.kr/B552015/NpsBplcInfoInqireService/getDetailInfoSearch'
cnt = 0

for code in tqdm(data_ins_1.seq.tolist()):
    params ={'serviceKey': api_key2, 'seq': code}
    response = requests.get(url, params=params)

    cc = xmltodict.parse(response.content)  # return collections.OrderedDict
    dd = json.loads(json.dumps(cc)) # return dict
    
    try:
        if dd['response']['header']['resultCode'] == '00':
            data_ins_2 = pd.concat([data_ins_2,
                                   pd.DataFrame([dd['response']['body']['item']])], axis=0)
    except:
        cnt += 1
        print(f'{cnt} {code}')
        
    time.sleep(0.1)

    
data_ins_2

100%|██████████████████████████████████████████████████████████████████████████████| 1986/1986 [07:44<00:00,  4.28it/s]


,adptDt,bzowrRgstNo,crrmmNtcAmt,jnngpCnt,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,scsnDt,vldtVlKrnNm,wkplIntpCd,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,20020401,135813****,14996040,48,41,550,360,00010101,플라스틱 창호 제조업,252400,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
0,20210104,653880****,909980,3,11,440,123,00010101,의료용 기구 소매업,523120,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
0,20140625,114871****,4803720,13,11,680,107,00010101,사업 및 무형 재산권 중개업,749903,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
0,19991125,410814****,83023960,301,29,170,129,00010101,주형 및 금형 제조업,292903,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
0,19880101,107811****,95653940,210,47,111,111,00010101,그 외 기타 1차 철강 제조업,271902,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20190604,111843****,979380,3,11,680,108,00010101,응용 소프트웨어 개발 및 공급업,722000,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
0,20140410,318810****,1089680,7,43,730,250,00010101,BIZ_NO미존재사업장,999999,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
0,20210426,433880****,405000,3,48,310,370,00010101,"미장, 타일 및 방수 공사업",452101,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
0,19880101,139810****,3945120,17,11,545,101,00010101,의약품 도매업,513311,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1


In [14]:
#data_ins_2.to_pickle('data_ins_2.pkl')
data_ins_2 = pd.read_pickle('data_ins_2.pkl')
data_ins_2

,adptDt,bzowrRgstNo,crrmmNtcAmt,jnngpCnt,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,scsnDt,vldtVlKrnNm,wkplIntpCd,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,20020401,135813****,14996040,48,41,550,360,00010101,플라스틱 창호 제조업,252400,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
0,20210104,653880****,909980,3,11,440,123,00010101,의료용 기구 소매업,523120,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
0,20140625,114871****,4803720,13,11,680,107,00010101,사업 및 무형 재산권 중개업,749903,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
0,19991125,410814****,83023960,301,29,170,129,00010101,주형 및 금형 제조업,292903,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
0,19880101,107811****,95653940,210,47,111,111,00010101,그 외 기타 1차 철강 제조업,271902,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20190604,111843****,979380,3,11,680,108,00010101,응용 소프트웨어 개발 및 공급업,722000,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
0,20140410,318810****,1089680,7,43,730,250,00010101,BIZ_NO미존재사업장,999999,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
0,20210426,433880****,405000,3,48,310,370,00010101,"미장, 타일 및 방수 공사업",452101,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
0,19880101,139810****,3945120,17,11,545,101,00010101,의약품 도매업,513311,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1


### 3. 국민연금 가입 사업장 내역 - 기간별 현황 정보조회
* 데이터가 거의 없거나 이상함 -> 기간별 흐름 보기에 한계가 있음.. 제외

In [366]:
data_ins_3 = pd.DataFrame()
url = 'http://apis.data.go.kr/B552015/NpsBplcInfoInqireService/getPdAcctoSttusInfoSearch'
cnt = 0

for code in tqdm(data_ins_1.seq.tolist()[:5]):
    for ym in ['202112', '202111', '202110', '202109', '202108', '202107', '202106',
               '202105', '202104', '202103', '202102', '202101']:
        params ={'serviceKey': api_key2, 'seq': code, 'data_crt_ym': ym}
        response = requests.get(url, params=params)

        cc = xmltodict.parse(response.content)  # return collections.OrderedDict
        dd = json.loads(json.dumps(cc)) # return dict

        print(code, dd, ym)
        '''
        try:
            if dd['response']['header']['resultCode'] == '00':
                data_ins_3 = pd.concat([data_ins_3,
                                       pd.DataFrame([dd['response']['body']['item']])], axis=0)
        except:
            cnt += 1
            print(f'{cnt} {code}')

        time.sleep(0.1)
        '''

    data_ins_3

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202112
32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202111
32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202110
32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202109
32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202108
32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'page

 20%|████████████████▊                                                                   | 1/5 [00:01<00:05,  1.26s/it]

32344013 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202101
33251682 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202112
33251682 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202111
33251682 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202110
33251682 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202109
33251682 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'page

 40%|█████████████████████████████████▌                                                  | 2/5 [00:02<00:03,  1.24s/it]

...

31966734 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202101
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'lssJnngpCnt': '10', 'nwAcqzrCnt': '12'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}} 202112
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202111
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202110
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202109
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, '

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:05<00:01,  1.46s/it]

36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202103
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202102
36603791 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202101
36654621 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'lssJnngpCnt': '3', 'nwAcqzrCnt': '4'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}} 202112
36654621 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202111
36654621 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'bo

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.42s/it]

36654621 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202103
36654621 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202102
36654621 {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': None, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '0'}}} 202101


In [25]:
# 법정동 코드

data_dong = pd.read_csv('법정동코드 전체자료.txt', sep='\t', encoding='cp949')
data_dong

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46309,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46310,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46311,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46312,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [26]:
# 시도 단위만 뽑기
data_dong = data_dong[data_dong.폐지여부 == '존재']
data_dong_rev = data_dong[data_dong.법정동명.str.split(' ').apply(lambda x: len(x)) == 1]
data_dong_rev

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
2615,2600000000,부산광역시,존재
2912,2700000000,대구광역시,존재
3280,2800000000,인천광역시,존재
3624,2900000000,광주광역시,존재
3870,3000000000,대전광역시,존재
4054,3100000000,울산광역시,존재
4302,3611000000,세종특별자치시,존재
4456,4100000000,경기도,존재
10993,4200000000,강원도,존재


In [28]:
#data_dong_rev.to_pickle('data_dong_rev.pkl')
data_dong_rev = pd.read_pickle('data_dong_rev.pkl')
data_dong_rev

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
2615,2600000000,부산광역시,존재
2912,2700000000,대구광역시,존재
3280,2800000000,인천광역시,존재
3624,2900000000,광주광역시,존재
3870,3000000000,대전광역시,존재
4054,3100000000,울산광역시,존재
4302,3611000000,세종특별자치시,존재
4456,4100000000,경기도,존재
10993,4200000000,강원도,존재
